# **2.B FEATURE EXTRACTION**
Phishing URLs only

#### The objective of this notebook is to collect data and save it as a CSV file for Feature Extraction.

* Lexical Features
* Whois Features
* Popularity Features

#### This project is worked on Jupyter Notebook 

In [1]:
import pandas as pd
from urllib.parse import urlparse
import re
from bs4 import BeautifulSoup
import whois
import urllib.request
import time
import socket
from urllib.error import HTTPError
from datetime import datetime

In [2]:
#import dataset

#/Users/jillkathleen/Downloads/github/masqueraderx-Phishing-Websites-Detection/ModelTrain/raw_datasets/1000-phishing.txt

#/Users/jillkathleen/Downloads/github/masqueraderx-Phishing-Websites-Detection/ModelTrain/raw_datasets/legitimate_urls.txt

legitimate = pd.read_csv("/Users/jillkathleen/Downloads/github/Detection-of-Phishing-Websites-using-an-Efficient-Machine-Learning-Framework-main/2_Feature_Extraction_of_Legitimate_URL/Benignurl_orignal.csv")



In [3]:
legitimate

,http://1337x.to/torrent/1048648/American-Sniper-2014-MD-iTALiAN-DVDSCR-X264-BST-MT/
0,http://1337x.to/torrent/1110018/Blackhat-2015-...
1,http://1337x.to/torrent/1122940/Blackhat-2015-...
2,http://1337x.to/torrent/1124395/Fast-and-Furio...
3,http://1337x.to/torrent/1145504/Avengers-Age-o...
4,http://1337x.to/torrent/1160078/Avengers-age-o...
...,...
35372,https://lastpass.com/signup2.php?ac=1&from_uri...
35373,https://lastpass.com/signup2.php?ac=1&from_uri...
35374,https://lastpass.com/signup2.php?ac=1&from_uri...
35375,https://lastpass.com/signup2.php?ac=1&from_uri...


In [4]:

legit = legitimate.sample(n = 6000, random_state = 12).copy()

legitimate_urls = legit.reset_index(drop=True)
legitimate_urls


,http://1337x.to/torrent/1048648/American-Sniper-2014-MD-iTALiAN-DVDSCR-X264-BST-MT/
0,http://graphicriver.net/search?date=this-month...
1,http://ecnavi.jp/redirect/?url=http://www.cros...
2,https://hubpages.com/signin?explain=follow+Hub...
3,http://extratorrent.cc/torrent/4190536/AOMEI+B...
4,http://icicibank.com/Personal-Banking/offers/o...
...,...
5995,http://mylust.com/videos/26185/extreme-pussy-i...
5996,http://onedio.com/haber/irak-savunma-bakanligi...
5997,http://io9.com/this-eerie-chandelier-is-contro...
5998,https://www.gov.uk/government/policies/local-e...


In [5]:
legitimate_urls.columns = ['url']
legitimate_urls.head()

,url
0,http://graphicriver.net/search?date=this-month...
1,http://ecnavi.jp/redirect/?url=http://www.cros...
2,https://hubpages.com/signin?explain=follow+Hub...
3,http://extratorrent.cc/torrent/4190536/AOMEI+B...
4,http://icicibank.com/Personal-Banking/offers/o...


## 2.1 Lexical Features

* URL Length 
* URL Shortening Services “TinyURL”
* URL Presence of "@" Symbol
* URL Presence of special characters : _ ? = & etc
* URL Suspicious words (security sensitive words)
* URL Digit Count
* URL Protocol Count (http / https)
* URL Dot Count
* URL Hyphen Count
* Domain presence of IP Address
* Domain presence of hyphen / prefix or Suffix
* Sub Domain and Multi Sub Domains Count
* Redirecting "//" in URL (// position)
* URL presence of EXE


In [6]:
#class FeatureExtraction:
#    def __init__(url):
#        pass

# 1.Extracts domain from the given URL
def getDomain(url):
    domain = urlparse(url).netloc
    if re.match(r"^www.",domain):
        domain = domain.replace("www.","")
    return domain
    
# 2.Checks for IP address in URL (Have_IP)
def ip_address(url):
    try:
        ipaddress.ip_address(url)
        ip = 1
    except:
        ip = 0
    return ip
    
# 3.Checks the presence of @ in URL (Have_At)
def have_at_symbol(url):
    if "@" in url:
        at = 1 
    else:
        at = 0   
    return at
    
# 4.Finding the length of URL and categorizing (URL_Length)
def long_url(url):
    if len(url) < 54:
        length = 0    
    else:
        length = 1    
    return length

# 5.Gives number of '/' in URL (URL_Depth)
def getDepth(url):
    s = urlparse(url).path.split('/')
    depth = 0
    for j in range(len(s)):
        if len(s[j]) != 0:
            depth = depth+1
    return depth
        
# 6.Checking for redirection '//' in the url (Redirection)
def redirection(url):
    pos = url.rfind('//')
    if pos > 6:
        if pos > 7:
            return 1
        else:
            return 0
    else:
        return 0
    
# 7.Existence of “HTTPS” Token in the Domain Part of the URL (https_Domain)
def httpDomain(url):
    domain = urlparse(url).netloc
    if 'https://|http://' in domain:
        return 1
    else:
        return 0

    
# 8. Checking for Shortening Services in URL (Tiny_URL) 
def shortening_service(url):
    match = re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                    'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                    'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                    'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                    'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                    'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                    'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                    'tr\.im|link\.zip\.net', url)
    if match:
        return 1               # phishing
    else:
        return 0               # legitimate
    
    
    
    
# 9.Checking for Prefix or Suffix Separated by (-) in the Domain (Prefix/Suffix)     
def prefix_suffix_separation(url):
    if "-" in urlparse(url).netloc:
        return 1            # phishing
    else:
        return 0            # legitimate
    
# 10. DNS Record 

    
# 11.Web traffic (Web_Traffic)
def web_traffic(url):
    try:
        url = urllib.parse.quote(url)
        rank = BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find(
        "REACH")['RANK']
        rank = int(rank)
    except TypeError:
        return 1
    if rank <100000:
        return 1
    else:
        return 0
        
# 12.Survival time of domain: The difference between termination time and creation time (Domain_Age)  
def domainAge(domain_name):
    creation_date = domain_name.creation_date
    expiration_date = domain_name.expiration_date
    if (isinstance(creation_date,str) or isinstance(expiration_date,str)):
        try:
            creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
            expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
        except:
            return 1
    if ((expiration_date is None) or (creation_date is None)):
        return 1
    elif ((type(expiration_date) is list) or (type(creation_date) is list)):
        return 1
    else:
        ageofdomain = abs((expiration_date - creation_date).days)
        if ((ageofdomain/30) < 6):
            age = 1
        else:
            age = 0
    return age

# 13.End time of domain: The difference between termination time and current time (Domain_End) 
def domainEnd(domain_name):
    expiration_date = domain_name.expiration_date
    if isinstance(expiration_date,str):
        try:
            expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
        except:
            return 1
    if (expiration_date is None):
        return 1
    elif (type(expiration_date) is list):
        return 1
    else:
        today = datetime.now()
        end = abs((expiration_date - today).days)
    if ((end/30) < 6):
        end = 0
    else:
        end = 1
    return end

# 14. Dot count
def dot_count(url):
    if url.count(".") < 3:
        return 0            # legitimate
    elif url.count(".") == 3:
        return 1            # suspicious
    else:
        return 1            # phishing
        
    
# 14. Special characters count
def specialcharCount(url):
    cnt = 0
    special_characters = [';','+=','_','?','=','&','[',']','/',':']
    for each_letter in url:
        if each_letter in special_characters:
            cnt = cnt + 1
    return cnt


# 15. 
def subdomCount(url):

    # separate protocol and domain then count the number of dots in domain
    
    domain = url.split("//")[-1].split("/")[0].split("www.")[-1]
    if(domain.count('.')<=1):
        return 0
    else:
        return 1

In [7]:
#Function to extract features
def featureExtraction(url,label):
    
    features = []
    
    features.append(getDomain(url))
    features.append(ip_address(url))
    features.append(have_at_symbol(url))
    features.append(long_url(url))
    features.append(getDepth(url))
    features.append(redirection(url))
    features.append(httpDomain(url))
    features.append(shortening_service(url))
    features.append(prefix_suffix_separation(url))
  
    dns = 0
    try:
        domain_name = whois.whois(urlparse(url).netloc)
    except:
        dns = 1
        
    features.append(dns)
    features.append(web_traffic(url))
    features.append(1 if dns == 1 else domainAge(domain_name))
    features.append(1 if dns == 1 else domainEnd(domain_name))
    
    features.append(dot_count(url))
    features.append(specialcharCount(url))
    features.append(subdomCount(url))
    

    features.append(label)
    
    
    return features

In [8]:
feature_names = ['domain', 'ip_present', 'at_present', 'url_length', 'url_depth','redirection', 
                      'https_domain', 'short_url', 'prefix/suffix', 'dns_record', 'web_traffic', 
                      'domain_age', 'domain_end', 'dot_count', 'specialchar_count','subdom_count', 'label']

label = 0

In [9]:
# Extracting the features & storing them in a list
# Lexical Features

# starting time
start_time = time.time()
print('\n')
print('Begin feature extraction for benign dataset.... \n')

##===================================##


#Extracting the feautres & storing them in a list
legit_features = []
rows = len(legitimate_urls['url'])
label = 0

for i in range(0, rows):
    url = legitimate_urls['url'][i]
    print(i), print(url)
    
    
    legit_features.append(featureExtraction(url,label))

    
##===================================##

elapsed = time.strftime("%H:%M:%S", time.gmtime(time.time() - start_time))
print('\n')
print(f"Runtime: Feature Extraction for legitimate dataset took:  {elapsed}")


print('\n\n\n\n')
print("***Legitimate Features")




Begin feature extraction for benign dataset.... 

0
http://graphicriver.net/search?date=this-month&length_max=&length_min=&price_max=&price_min=&rating_min=&sales=&sort=sales&term=&view=list
1
http://ecnavi.jp/redirect/?url=http://www.cross-a.net/x.php?id=1845_3212_22061_26563&m=1004&pid=%user_id%
2
https://hubpages.com/signin?explain=follow+Hubs&url=%2Fhub%2FComfort-Theories-of-Religion
3
http://extratorrent.cc/torrent/4190536/AOMEI+Backupper+Technician+%2B+Server+Edition+2.8.0+%2B+Patch+%2B+Key+%2B+100%25+Working.html
4
http://icicibank.com/Personal-Banking/offers/offer-detail.page?id=offer-ezeego-domestic-airtravel-20141407112611060
5
http://nypost.com/2015/05/07/us-indifference-leaves-saudis-partnering-with-terrorists/
6
http://kienthuc.net.vn/diem-thi/diem-chuan-dh-cong-nghe-giao-thong-van-tai-nam-2014-482407.html
Error trying to connect to socket: closing socket
7
http://thenextweb.com/in/2015/04/16/india-wants-a-neutral-web-and-facebooks-internet-org-cant-be-a-part-of-it/gtm.j

71
https://paytm.com/blog/paytm-offer-for-app-users-get-upto-rs-50-cash-back/?share=email
72
http://motthegioi.vn/suckhoe/nu-cuoi-y-hoc/nu-cuoi-suc-khoe-nhung-li-do-nen-yeu-chang-trai-bung-bu-188294.html
Error trying to connect to socket: closing socket
73
http://torcache.net/torrent/047D47DFF4DC5CD9BEA6D0F4C57D68F2F2D71205.torrent?title=[kickass.to]night.at.the.museum.secret.of.the.tomb.2014.1080p.brrip.x264.yify
74
http://thenextweb.com/asia/2014/09/26/myanmars-mobile-revolution-kicks-telenor-prepares-launch-service/gtm.start
75
http://extratorrent.cc/torrent/4189616/Jedi+Mind.Tricks.The.Thief.and.the.Fallen.2015.mp3.vbr.NOiR.html
76
http://genius.com/2990084/Us-general-services-administration-gsa-mentor-protege-program-subpart-51970/Incentives-for-prime-contractors
77
http://thenextweb.com/apps/2012/04/19/500px-launches-android-app-and-overhauls-its-ipad-version-too/
78
http://kienthuc.net.vn/tin-tuc-tuyen-sinh/ty-le-choi-cua-dai-hoc-fpt-nam-2014-487408.html
Error trying to connect 

138
http://extratorrent.cc/torrent/4189415/Snow%2C+Heather-Sweet+Deception+-+epub+-+zeke23.html
139
http://torcache.net/torrent/4EFE4986FC8B15951469CF6AC82FE102E8C9007A.torrent?title=[kickass.to]if.i.stay.2014.720p.brrip.x264.yify
140
https://twitter.com/share?url=http%3A%2F%2Fhubpages.com%2Fhub%2FEMT-Electrical-Conduit-Pipe-Bending-the-Math-Behind-a-Conduit-Bending-Guide&text=Formulas+and+Multipliers+For+Bending+Conduit+or+Electrical+Pipe
141
http://seekingalpha.com/article/2471615-mexico-economy-to-move-with-energy-reforms-part-1
142
http://web.tv/liveCategory/6/language/1/index/changeLanguage/newshared/searchAutoComplete
143
http://indianexpress.com/article/sports/football/chelsea-go-past-leicester-city-3-1-eye-premier-league-title/
144
http://web.de/magazine/sport/fussball/champions-league/wunder-fc-schalke-04-real-madrid-champions-league-17321480
145
http://olx.co.id/i2/elektronik-gadget/komputer/keyboard-mouse/elektronik-gadget/komputer/keyboard-mouse
146
http://mylust.com/videos

207
http://persianblog.ir/tags/1082/12/%d8%a7%d9%86%d8%aa%d8%ae%d8%a7%d8%a8%d8%a7%d8%aa/
208
http://onedio.com/haber/bir-gunlugune-super-kahraman-oldun-bakalim-hayatta-kalabilecek-misin--504266
209
http://hollywoodlife.com/2015/05/04/ian-somerhalder-nikki-reed-double-date-paul-wesley-phoebe-tonkin-vampire-diaries/
210
http://fortune.com/2015/04/02/pfizer-to-stop-selling-vaccines-in-china-after-failing-to-get-license-renewal/
211
http://ringring.vn/dat-hong-dang-va-kang-tae-oh-tuoi-thanh-xuan-len-ban-can-114271.html
Error trying to connect to socket: closing socket
212
http://nguyentandung.org/dieu-gi-xay-ra-neu-chung-ta-khong-xu-ly-kheo-vu-gian-khoan-981.html
213
http://buzzfil.net/article/5466/animaux/georges-le-chat-qui-se-tient-debout-tout-le-temps-3.html?href=inner_website
214
http://thenextweb.com/insider/2014/10/29/unbabel-integrates-mailchimp-offer-translation-service-promotional-emails/gtm.js/
215
http://fanpage.gr/must-watch/%ce%b4%ce%b5%ce%af%cf%84%ce%b5-%cf%84%ce%b9%cf%82-%c

270
http://mirtesen.ru/url?e=simple_click&blog_post_id=43711607498&url=http%3A%2F%2Fchampionat.mirtesen.ru%2Fblog%2F43711607498%2FDenisenko-vyishel-v-chetvertfinal-chempionata-mira-po-thekvondo-
271
http://indianexpress.com/article/india/india-others/salman-khan-gets-relief-bombay-hc-suspends-his-5-year-sentence-in-hit-and-run-case/
272
https://medium.com/@nathancoppedge/excellent-article-genius-typology-4e1a8317457e?source=has-recommended
273
http://patch.com/california/petaluma/breaking-news-daughters-save-mothers-life-after-learning-cpr-through-heartsafe-program/
274
http://popupads.ir/blog/?tag=%d8%a8%d9%87%d8%aa%d8%b1%db%8c%d9%86-%d8%b3%d8%a7%db%8c%d8%aa-%d9%be%d8%a7%d9%be-%d8%a2%d9%be
275
http://grantland.com/hollywood-prospectus/three-potential-replacements-for-the-ousted-piers-morgan-hint-theyre-all-jay-leno/
276
https://twitter.com/home?status=%E3%83%8C%E3%81%91%E3%82%8B%EF%BC%81%E3%80%90%E5%B7%A8%E4%B9%B3%E3%80%91+http%3A%2F%2Fero-video.net%2Ft%2FInoAwxva2pEyzD9f+%E7%BE%8E%E5

...

5698
http://udn.com/news/story/7340/892813-%E7%8E%8B%E5%81%A5%E5%A3%AF%EF%BC%8F%E4%B8%80%E5%80%8B%E5%90%8D%E5%AD%97%E7%9A%84%E6%88%B0%E7%88%AD
5699
http://superuser.com/questions/913706/is-it-possible-to-run-nvidia-geforce-gtx-970-g1-gaming-on-asrock-870-extreme3-r2
5700
http://bdnews24.com/aviation/2015/04/29/shape-changing-wings-are-future-of-aviation-nasa
5701
http://codecanyon.net/item/fives-full-ios78-multi-language-game-template-swift/11230359
5702
http://himado.in/?keyword=%E6%A9%9F%E5%8B%95%E6%88%A6%E8%89%A6%E3%83%8A%E3%83%87%E3%82%B7%E3%82%B3
5703
http://thechive.com/2015/03/11/charlotte-mckinney-wasnt-meant-for-clothing-24-photos/
5704
http://motthegioi.vn/nhung-chien-binh-vang/ky-6-nguoi-my-khong-tram-lang-103327.html
Error trying to connect to socket: closing socket
5705
http://techcrunch.com/2015/01/11/this-desk-will-tell-you-when-to-sit-and-when-to-stand/
5706
http://bestblackhatforum.com/Thread-Fully-Approved-Adsense-Account-s?action=newpost
5707
http://udn.com/news/stor

5765
http://distractify.com/post/related/id/55524aad4a0c4b5153d5da6b/skip/40/limit/10/back/0
5766
http://indianexpress.com/article/cities/mumbai/send-more-doctors-to-tribal-areas-high-court-tells-state/
5767
http://grantland.com/features/how-universal-soldier-series-became-weirdest-most-indestructible-film-franchise-our-time/
5768
http://pornsharing.com/apple-ass-latina-eva-saldana-cleans-the-kitchen-naked_v78610
5769
http://otomoto.pl/oferta/case-ih-527-claas-fendt-new-holland-case-john-deere-ID6y3PhH.html
5770
http://tinnhanh360.net/cl-ip-tinh-cam-cua-dai-ca-thai-lan-gay-sot-cong-dong-mang.html
5771
http://katproxy.com/blog/Bayfia/post/i-learn-something-new-everyday-but-not-all-news-is-good-news/
5772
http://teespring.com/en-AU/atp-shirt-sport-men?utm_campaign=atp-shirt-sport-men&utm_medium=oembed&utm_source=wordpress
5773
http://mic.com/articles/115692/the-police-shooting-of-44-year-old-eric-harris-just-took-a-horrifying-turn
5774
http://babal.net/books/view/1190/%D9%88%D8%AD%D9%8A-

5837
http://allrecipes.com/video/650/best-steak-marinade-in-existence/detail.aspx?prop24=VH_Popular
5838
http://motthegioi.vn/goc-nhin-khac/dung-tro-thanh-nguoi-ky-thi-dong-tinh-167641.html
Error trying to connect to socket: closing socket
5839
http://nrk.no/hurtigruten/js/src/3rdParty/jquery.fancybox-1.3.4/fancybox/jquery.fancybox-1.3.4.pack.js
5840
http://jalopnik.com/ten-old-car-video-games-that-are-still-great-today-1701489705?utm_source=recirculation&utm_medium=recirculation&utm_campaign=fridayPM
5841
http://digg.com/video/its-slightly-terrifying-how-much-this-bird-sounds-like-a-computer
5842
http://digg.com/video/what-happens-when-a-soccer-players-uses-a-wrestling-move-on-his-teammate
5843
http://venturebeat.com/2015/05/13/google-blocks-extensions-not-listed-in-the-chrome-web-store-for-all-windows-users-will-expand-to-mac-in-july/
5844
http://buzzfil.net/article/5692/non-classe/la-recette-parfaite-pour-avoir-un-ventre-plat-9.html?href=inner_website
5845
http://tinnhanh360.net/vdc

5904
http://censor.net.ua/news/335871/kerri_ssha_schitayut_neobhodimym_polnoe_prekraschenie_ognya_v_shirokino
5905
http://seasonvar.ru/serial-189-ZHenskij_Klub_po_rassledovaniyu_ubijstv-1-season.html
5906
http://codecanyon.net/search?date=this-month&length_max=&length_min=&platform=WordPress&price_max=&price_min=&rating_min=&sales=&sort=sales&term=&utf8=%E2%9C%93&view=list
5907
https://ifttt.com/recipes/24250-camera-less-at-sxsw-scrapbook-everyone-else-s-instagrams-instead
5908
http://correios.com.br/sobre-correios/a-empresa/publicidade-1/campanhas-publicitarias-dos-correios/imagens/CampanhaSEDEX30Anos.jpg
5909
http://mylust.com/videos/66863/indonesian-chubby-whore-prima-has-the-biggest-tits-i-d-ever-seen/
5910
http://distractify.com/post/related/id/554526ab4a0c4bf1609ba3c7/skip/10/limit/10/back/0
5911
http://kenh14.vn/sport/ngoi-sao-u21-viet-nam-tiec-nuoi-vi-lo-hen-sea-games-20150427095622645.chn
Error trying to connect to socket: closing socket
5912
http://tobogo.net/cdsb/board.php?b

5966
https://quizlet.com/sign-up?redir=https%3A%2F%2Fquizlet.com%2Flogin%3Fredir%3D%252F6396803%252Frecreate-set%252Fnew
5967
http://motthegioi.vn/khoa-hoc-giao-duc/blog-sinh-vien/sinh-vien-o-tro-can-nhung-vat-dung-gi-176887.html
Error trying to connect to socket: closing socket
5968
http://mic.com/articles/117138/17-dramatic-photos-show-the-violent-may-day-protests-that-rocked-istanbul
5969
http://censor.net.ua/photo_news/335935/terroristy_stremyatsya_sorvat_peremirie_styagivayut_tyajeloe_voorujenie_i_uvelichivayut_kolichestvo
5970
http://extratorrent.cc/torrent/4190806/Hacker+4+-+Nuove+tecniche+di+protezione+%5BPDF+ITA%5D.html
5971
http://thenextweb.com/entrepreneur/2014/02/25/best-entrepreneurs-dont-start-companies-invent-categories/gtm.js
5972
http://icicibank.com/Personal-Banking/account-deposit/savings-account/moneymultiplier.page
5973
http://nguyentandung.org/chay-dua-tau-ngam-khu-vuc-nga-tiet-lo-phuong-phap-tac-chien-cho-viet-nam.html
5974
http://bleacherreport.com/articles/231

In [12]:
#Converting the list to dataframe

legitimate = pd.DataFrame(legit_features, columns= feature_names)
legitimate.head()

,domain,ip_present,at_present,url_length,url_depth,redirection,https_domain,short_url,prefix/suffix,dns_record,web_traffic,domain_age,domain_end,dot_count,specialchar_count,subdom_count,label
0,graphicriver.net,0,0,1,1,0,0,0,0,0,1,1,1,0,29,0,0
1,ecnavi.jp,0,0,1,1,1,0,0,0,0,1,1,1,1,21,0,0
2,hubpages.com,0,0,1,1,0,0,0,0,0,1,0,1,0,8,0,0
3,extratorrent.cc,0,0,1,3,0,0,0,0,0,0,0,1,1,6,0,0
4,icicibank.com,0,0,1,3,0,0,0,0,0,1,0,1,0,8,0,0


In [13]:
# Storing the extracted legitimate URLs fatures to csv file

legitimate.to_csv('/Users/jillkathleen/Desktop/Thesis 2/Phishing Analysis Detection/Extracted CSVs/from FeatureExtraction/D3_benign_updated.csv', index= False)

